# SolarMED model

Model that integrates all the component models and subproblems into one interface.

This notebook is used to develop and test it

In [1]:
from pathlib import Path
import time
import datetime
import numpy as np
import pandas as pd
from iapws import IAPWS97 as w_props
import hjson
from loguru import logger

# Visualization packages
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

from solarmed_modeling.utils import data_preprocessing, data_conditioning

# auto reload modules
%load_ext autoreload
%autoreload 2

# Paths definition
output_path: Path = Path("../../docs/models/attachments")
data_path: Path = Path("../data")

date_str: str = "20230703" # '20230630'
filename_process_data = f'{date_str}_solarMED.csv'
filename_process_data2 = f'{date_str}_MED.csv'

# Available data to test
# data/calibration/20230807_aquasol.csv
# data/calibration/20230707_20230710_datos_tanques.csv
# Nextcloud/Juanmi_MED_PSA/EURECAT/data/20231030_solarMED.csv

sample_rate = '400s'
sample_rate_numeric = int(sample_rate[:-1])

# Resample figures using plotly_resampler
resample_figures: bool = False

# Parameters
cost_w: float = 3 # €/m³, cost of water
cost_e: float = 0.05 # €/kWh, cost of electricity


### Pre-processing

In [2]:
data_paths = [data_path / f"datasets/{filename_process_data}", data_path / f"datasets/{filename_process_data2}"]

with open( data_path / "variables_config.hjson") as f:
    vars_config = hjson.load(f)
    
with open(data_path/"plot_config.hjson") as f:
    plot_config = hjson.load(f)
    
# Load data and preprocess data
df = data_preprocessing(data_paths, vars_config, sample_rate_key=sample_rate)

# Condition data
df = data_conditioning(df, cost_w=cost_w, cost_e=cost_e, sample_rate_numeric=sample_rate_numeric, vars_config=vars_config)


2024-11-13 11:10:51.599 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:81 - Signal id PK-MED-E03-pa not found in dataframe columns.
2024-11-13 11:10:51.600 | WARNING  | phd_visualizations.utils:rename_signal_ids_to_var_ids:81 - Signal id FT-SF-003 not found in dataframe columns.
2024-11-13 11:10:51.610 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tamb to C from C
2024-11-13 11:10:51.611 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_c_in to C from C
2024-11-13 11:10:51.612 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_s_in to C from C
2024-11-13 11:10:51.613 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_s_out to C from C
2024-11-13 11:10:51.614 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tmed_c_out to C from C
2024-11-13 11:10:51.616 | DEBUG    | phd_visualizations.utils.units:unit_conversion:552 - Updated Tsf_i

### Test visualization

In [67]:
# df.to_csv('data/dev.csv')


In [ ]:
# Update plot config
with open( data_path / "plot_config.hjson") as f:
    plot_config = hjson.load(f)

fig = experimental_results_plot(plot_config, df, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)


### Model evaluation

In [9]:
# Test model
import math
from solarmed_modeling.solar_med import SolarMED

logger.enable("solarmed_modeling")

span = math.ceil(600 / sample_rate_numeric) # 600 s
idx_start = np.max([span, 2]) # idx_start-1 should at least be one 
idx_end = len(df)
df_mod = pd.DataFrame()

# Initialize model
model = SolarMED(
    resolution_mode='constant-water-props',
    use_models=True,
    use_finite_state_machine=True,
    
    sample_time=sample_rate_numeric,
    
    # Initial states
    ## Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[idx_start], df['Tts_h_m'].iloc[idx_start], df['Tts_h_b'].iloc[idx_start]], 
    Tts_c=[df['Tts_c_t'].iloc[idx_start], df['Tts_c_m'].iloc[idx_start], df['Tts_c_b'].iloc[idx_start]],
    
    ## Solar field
    Tsf_in_ant=df['Tsf_in'].iloc[idx_start-span:idx_start].values,
    qsf_ant=df['qsf'].iloc[idx_start-span:idx_start].values,
    
    # cost_w = 3, # €/m³ 
    # cost_e = 0.05, # €/kWhe,
)

# Save model initial state and configuration
model_config = model.model_dump_configuration()
df_mod = model.to_dataframe(df_mod)
# model_dump = model.model_dump()

# Run model
for idx in range(idx_start+1, idx_end):
    ds = df.iloc[idx]
    
    # logger.info(f"Iteration {idx} / {idx_end}")
    start_time = time.time()
        
    model.step(
        # Decision variables
        ## MED
        qmed_s=ds['qmed_s'],
        qmed_f=ds['qmed_f'],
        Tmed_s_in=ds['Tmed_s_in'],
        Tmed_c_out=ds['Tmed_c_out'],
        ## Thermal storage
        qts_src=ds['qhx_s'],
        ## Solar field
        qsf=ds['qsf'],
        
        med_vacuum_state=2,
                
        # Environment variables
        Tmed_c_in=ds['Tmed_c_in'],
        Tamb=ds['Tamb'],
        I=ds['I'],
    )
    
    logger.info(f"Finished Iteration {idx} / {idx_end} - {df.index[idx]:%H:%M:%S}, elapsed time: {time.time()-start_time:.2f} seconds.")

    df_mod = model.to_dataframe(df_mod)


2024-11-13 11:23:08.264 | INFO     | solarmed_modeling.solar_med:init_matlab_engine:1413 - MATLAB engine initialized
2024-11-13 11:23:08.274 | INFO     | solarmed_modeling.solar_med:model_post_init:539 - 
        SolarMED model initialized with: 
            - Evaluating models: True
            - Evaluating finite state machines: True
            - Resolution mode: constant-water-props
            - On limits violation policy: clip
            - Sample time: 400.0 s
            - MED actuators: ['med_brine_pump', 'med_feed_pump', 'med_distillate_pump', 'med_cooling_pump', 'med_heatsource_pump']
            - Solar field actuators: ['sf_pump']
            - Thermal storage actuators: ['ts_src_pump']
            - Model parameters: ModelParameters(sf=ModelParameters(beta=0.0436396, H=13.676448551722462, gamma=0.1), ts=ModelParameters(UA_h=(0.0069818, 0.00584034, 0.03041486), V_h=(5.94771006, 4.87661781, 2.19737023), UA_c=(0.01396848, 0.0001, 0.02286885), V_c=(5.33410037, 7.56470594, 0.9

In [ ]:
# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx<idx_end-1 else idx_end] # idx_start-1 because now we are adding one element after the initialization

# Update plot config
with open( data_path / "plot_config_validation.hjson") as f:
    plt_config = hjson.load(f)

fig = experimental_results_plot(plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)


In [56]:
# Save figure
# save_figure(
#     figure_name=f"SolarMED_validation_{df.index[0].strftime('%Y%m%d')}", 
#     figure_path=output_path,
#     fig=fig, formats=('png', 'html'), 
#     width=fig.layout.width, height=fig.layout.height, scale=2
# )


### Model evaluation using `evaluate_model`

In [ ]:
from solarmed_modeling.solar_med.utils import evaluate_model
from solarmed_modeling.solar_med import ModelParameters, FixedModelParameters

logger.enable('solarmed_modeling.solar_med.utils')

dfs_mod, stats = evaluate_model(
    df=df, sample_rate = sample_rate_numeric,
    model_params = ModelParameters(), 
    fixed_model_params = FixedModelParameters(),
    alternatives_to_eval = ["constant-water-props"],
)

fig = experimental_results_plot(plot_config, df, df_comp=dfs_mod[0], vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)
